In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# Read in shapefile and examine data
kerala = gpd.read_file('maps/kerala.json')
kerala.head()

,id,dt_code,district,st_code,year,st_nm,geometry
0,None,588,Kasaragod,32,2011_c,Kerala,"POLYGON ((75.41444 12.28996, 75.39198 12.29033..."
1,None,590,Wayanad,32,2011_c,Kerala,"POLYGON ((75.93655 11.94096, 75.93692 11.94170..."
2,None,591,Kozhikode,32,2011_c,Kerala,"POLYGON ((75.79406 11.78962, 75.78338 11.75973..."
3,None,592,Malappuram,32,2011_c,Kerala,"POLYGON ((76.14275 11.45071, 76.18583 11.47275..."
4,None,593,Palakkad,32,2011_c,Kerala,"POLYGON ((76.06911 10.73028, 76.06837 10.73290..."


In [3]:
datafile = 'csvs/kerala.csv'

In [4]:
df = pd.read_csv(datafile, names = ['district', 'R', 'Rproj'], skiprows = 1)
df.head()

,district,R,Rproj
0,Alappuzha,2.194601,3.867043
1,Ernakulam,1.000000,1.000000
2,Idukki,NaN,NaN
3,Kannur,3.059031,5.941673
4,Kasaragod,0.038613,-1.307328


In [5]:
merged = kerala.merge(df, on = 'district', how = 'left')
merged.head()

,id,dt_code,district,st_code,year,st_nm,geometry,R,Rproj
0,None,588,Kasaragod,32,2011_c,Kerala,"POLYGON ((75.41444 12.28996, 75.39198 12.29033...",0.038613,-1.307328
1,None,590,Wayanad,32,2011_c,Kerala,"POLYGON ((75.93655 11.94096, 75.93692 11.94170...",0.989879,0.975709
2,None,591,Kozhikode,32,2011_c,Kerala,"POLYGON ((75.79406 11.78962, 75.78338 11.75973...",0.179942,-0.968138
3,None,592,Malappuram,32,2011_c,Kerala,"POLYGON ((76.14275 11.45071, 76.18583 11.47275...",1.634256,2.522214
4,None,593,Palakkad,32,2011_c,Kerala,"POLYGON ((76.06911 10.73028, 76.06837 10.73290...",4.874815,10.299556


In [6]:
import json
#Read data to json.
merged_json = json.loads(merged.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)

In [7]:
from bokeh.io import output_notebook, show, output_file, save
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool
from bokeh.palettes import brewer

In [8]:
#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)
#Define a sequential multi-hue color palette.
palette = brewer['Blues'][7]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 14)
#Define custom tick labels for color bar.
tick_labels = {'0': '0','3': '3', '6': '6', '9': '9', '12': '12', '15': '15'}

In [9]:
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

In [ ]:
current = figure(title = 'R by district', plot_height = 900 , plot_width = 600, toolbar_location = None)
current.xgrid.grid_line_color = None
current.ygrid.grid_line_color = None

In [ ]:
cdistricts = current.patches('xs','ys', source = geosource,fill_color = {'field' :'R', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

In [ ]:
current.add_tools(HoverTool(renderers = [cdistricts],
                      tooltips = [('District','@district'),
                               ('Current R', '@R')]))
current.add_layout(color_bar, 'below')
#output_notebook()
show(current)
output_file('kerala_current.html')
save(current)

In [10]:
proj = figure(title = 'R by district', plot_height = 900 , plot_width = 600, toolbar_location = None)
proj.xgrid.grid_line_color = None
proj.ygrid.grid_line_color = None
pdistricts = proj.patches('xs','ys', source = geosource,fill_color = {'field' :'Rproj', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

proj.add_tools(HoverTool(renderers = [pdistricts],
                      tooltips = [('District','@district'),
                               ('Projected R', '@R')]))
proj.add_layout(color_bar, 'below')
output_file('kerala_projected.html')
save(proj)

'/Users/ananyakaranam/Desktop/India_Map/kerala_projected.html'